In [10]:
import logging 
import os
import pandas as pd 
import numpy as np 
import pandas_ta as ta
import requests

from datetime import datetime, timedelta

# All the Alpaca Loaders
from alpaca.data.historical import StockHistoricalDataClient, CryptoHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetAssetsRequest
from alpaca.trading.models import Asset, Order
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.enums import AssetClass
from dotenv import load_dotenv

dotenv_path = r'C:\Users\srerr\Documents\Projects\PersonalProjects\stonks\alpaca\.env'

token = '5781019360:AAHHE8_MV4qpgZE6U56o-otnhwZ2kVevNYQ'
chat_id = 1634990243

In [14]:
def send_telegram_message(message: str, bot_token: str = token, chat_id: int = chat_id):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage?chat_id={chat_id}&text={message}"

    try:
        response = requests.get(url)

        if not response:
            print(response.status_code)

    except Exception as e:
        print(e)

In [11]:
load_dotenv(dotenv_path=dotenv_path)
api_key = os.environ['ALPACA_PAPER_TRADING_KEY_ID']
secret_key = os.environ['ALPACA_PAPER_TRADING_SECRET_KEY']

trading_client = TradingClient(api_key=api_key, secret_key=secret_key)
stock_data_client = StockHistoricalDataClient(api_key=api_key, secret_key=secret_key)

In [12]:
markets = pd.read_csv(r'C:\Users\srerr\Documents\Projects\PersonalProjects\stonks\alpaca\data\alpaca_equity_markets.csv')

In [15]:
# Generating the request for market data
stock_symbol = 'AAPL'
start_date = datetime.today() - timedelta(days = 100)
market_data_request = StockBarsRequest(
    symbol_or_symbols=stock_symbol, 
    start = start_date, 
    timeframe=TimeFrame.Day
)

msg = f'Getting historical bars for {stock_symbol}'
send_telegram_message(message=msg)
historical_bars = stock_data_client.get_stock_bars(market_data_request).df
msg = f'Retrieved historical bars for {stock_symbol}'
send_telegram_message(message=msg)

In [16]:
long_sma = 40
short_sma = 10

long_sma_column_mame = f'{long_sma}-day SMA'
short_sma_column_name = f'{short_sma}-day SMA'

historical_bars[long_sma_column_mame] = historical_bars['close'].rolling(long_sma).mean() 
historical_bars[short_sma_column_name] = historical_bars['close'].rolling(short_sma).mean()
moving_average_signal = historical_bars[short_sma_column_name].iloc[-1] > historical_bars[long_sma_column_mame].iloc[-1]

if moving_average_signal == True:
    msg = "The 10-day SMA is above the 40-day SMA, we should be long"
else: 
    msg = 'The 10-day SMA is below the 40-day SMA, we should be short'

send_telegram_message(message = msg) 


In [18]:
all_orders = trading_client.get_orders()
current_asset_open_orders = [i for i in all_orders if i.symbol == stock_symbol]

In [19]:
# So we have the signal, we have to see if this is long or short
capital = 4_000
shares_amount_to_trade = capital // historical_bars['vwap'].iloc[-1]
positions = trading_client.get_all_positions()
current_positions_for_ticker = [i for i in positions if i.symbol == stock_symbol] 
if moving_average_signal == True: 
    trade_side = OrderSide.BUY
    side = 'buy'
else: 
    trade_side = OrderSide.SELL
    side = 'sell'

if len(current_asset_open_orders) != 0: 
    msg = f"We have some open orders for {stock_symbol} right now. Closing them ..."
    send_telegram_message(message=msg)

    trading_client.cancel_orders()

if len(current_positions_for_ticker) != 0:
    # We do a check to see if a position is open. If it is, we close it
    msg = f"We currently have a position open for {stock_symbol}. Closing it now..."
    send_telegram_message(msg)
    trading_client.close_position(symbol_or_asset_id=stock_symbol)
    msg = f'the position is closed'
    send_telegram_message(msg)


market_order_request = MarketOrderRequest(
    symbol = stock_symbol, 
    qty = shares_amount_to_trade, 
    side = trade_side, 
    time_in_force = TimeInForce.DAY
)

market_order = trading_client.submit_order(order_data = market_order_request)
market_order_msg = f'Sending a {side} order for {shares_amount_to_trade} shares of {stock_symbol}.'
send_telegram_message(message=market_order_msg)

In [26]:
market_order.filled_avg_price

In [22]:
class MovingAverageCrossOverStrategy:


    def __init__(self, symbol: str, short_period: int, long_period: int, alpaca_api_key: str, alpaca_secret_key: str, capital: float) -> None:

        self.capital = capital
        self.symbol = symbol
        self.short_period = short_period
        self.long_period = long_period
        self.alpaca_api_key = alpaca_api_key
        self.alpaca_secret_key = alpaca_secret_key
        self.positions = []
        self.trades = []
        self.total_pnl = 0

        self.trading_client = TradingClient(api_key=alpaca_api_key, secret_key=alpaca_secret_key)
        self.stock_data_client = StockHistoricalDataClient(api_key=alpaca_api_key, secret_key=alpaca_secret_key)
        self.crypto_data_client = CryptoHistoricalDataClient
        self.start_date = datetime.now()
        self.end_date = None
        self.historical_data = self.get_historical_data()


    def get_historical_data(self):
        data_start_time = self.start_date - timedelta(days = self.long_period + 100)
        market_data_request = StockBarsRequest(
            symbol_or_symbols=self.symbol,
            start = data_start_time,
            timeframe=TimeFrame.Day
        )

        df = self.stock_data_client.get_stock_bars(market_data_request).df

        return df

    def generate_signal(self):
        """Takes the historical data and generates a buy, sell, or hold signal

        Returns:
            str: Signal which tells us what to do based on the data and current position
        """

        historical_bars = self.get_historical_data()

        # Generating the moving averages
        long_sma_column_mame = f'{self.long_period}-day SMA'
        short_sma_column_name = f'{self.short_period}-day SMA'

        historical_bars[long_sma_column_mame] = historical_bars['close'].rolling(self.long_period).mean()
        historical_bars[short_sma_column_name] = historical_bars['close'].rolling(self.short_period).mean()

        # We want to check for a crossover, only then should we go short or long
        # 1. Shorter moving average crosses above the longer one
        # 2. Shorter moving average crosses below the longer one

        prev_short_moving_average = historical_bars[short_sma_column_name].iloc[-2]
        current_short_moving_average = historical_bars[short_sma_column_name].iloc[-1]

        prev_long_moving_average = historical_bars[long_sma_column_mame].iloc[-2]
        current_long_moving_average = historical_bars[long_sma_column_mame].iloc[-1]

        if (prev_short_moving_average > prev_long_moving_average) and (current_short_moving_average < current_long_moving_average):
            ## We have a cross going down and we should go short

            signal = OrderSide.SELL

        elif (prev_short_moving_average < prev_long_moving_average) and (current_short_moving_average > current_long_moving_average):

            # We have a cross going up and we should go long
            signal = OrderSide.BUY

        else:

            signal = None

        return signal

    def calculate_shares(self):
        assert not self.historical_data.empty(), "Historical data has not been run yet?"
        shares = self.capital // self.historical_data['vwap'].iloc[-1]

        return shares

    def run(self):
        """Goes through the process of running the strategy and everything involved
        """

        signal = self.generate_signal()
        shares_to_trade = self.calculate_shares()

        if signal == OrderSide.BUY or signal == OrderSide.SELL:
            if len(self.positions) == 0:

                market_order_request = MarketOrderRequest(
                    symbol = self.symbol,
                    qty = shares_to_trade,
                    side = signal,
                    time_in_force = TimeInForce.DAY
                )

                order = self.trading_client.submit_order(market_order_request)

                trade = AlpacaTrade(
                    symbol=self.symbol, 
                    side = signal, 
                    genesis_order_id=order.client_order_id, 
                    quantity = int(order.qty), 
                    fill_price=order.filled_avg_price
                )

                self.trades.append(trade)

            else:
                current_position = self.positions[0]
                amount_to_close = current_position.quantity

                if (current_position.side == 'Long' and signal == OrderSide.SELL) or (current_position.side == "Short" and signal == OrderSide.BUY):
                    # In this case, we have to close our long position and go short

                    market_order_close_out = self.create_and_submit_market_order(
                        quantity=amount_to_close,
                        side = signal
                    )



                    market_order_entry = self.create_and_submit_market_order(quantity=amount_to_close, side = signal)

                    #TODO Make sure to interact with the new alpaca position information


    def create_and_submit_market_order(self, quantity: int, side: OrderSide) -> Order:
        """Creates and submits a market order and returns the order object for you

        Args:
            quantity (int): Quantity to trade, either shares or tokens
            side (OrderSide): Orderside which you want to sell

        Returns:
            Order: Order object
        """

        request_params = MarketOrderRequest(
            symbol = self.symbol,
            qty = quantity,
            side = side,
            time_in_force = TimeInForce.DAY
        )

        order = self.trading_client.submit_order(order_data = request_params)

        return order

In [28]:
class AlpacaTrade: 


    def __init__(self, symbol: str, side: str, genesis_order_id: str, quantity: float, fill_price: float) -> None:
        self.symbol = symbol 
        self.side = side 
        self.quantity = quantity
        self.genesis_order_id = genesis_order_id
        self.fill_price = fill_price

        self.is_open = True

    def close_trade(self): 
        self.is_open = False

    def calc_dollar_pnl(self, current_price: float): 
        return (self.fill_price - current_price) * self.quantity

    def calc_perc_pnl(self, current_price: float): 

        return (self.fill_price - current_price)/(self.fill_price)

    @staticmethod 
    def calc_pnl(p1: float, p2: float): 
        return (p2 - p1) / p1 
    

    
